In [15]:
# Jupyter Notebook libraries
from IPython.display import display
from ipywidgets import widgets

# Python native libraries
import datetime as dt
from path import Path
from time import sleep
import sqlalchemy

# Conda libraries
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from bokeh.models import HoverTool

# External libraries
import mplfinance as mpf
import hvplot.pandas

# Local libraries
from utils.alpaca import alpaca as alp



In [16]:
active_assets = alp.list_assets(status='active')
active_assets_df = pd.DataFrame([asset._raw for asset in active_assets if asset._raw['tradable'] == True])
active_assets_df = active_assets_df.loc[lambda df: (df['class'] == 'us_equity')]
active_assets_df.drop(columns=['id', 'name', 'class', 'status', 'tradable', 'min_order_size', 'min_trade_increment', 'price_increment'], inplace=True)
active_assets_df.drop(columns=['marginable', 'shortable', 'easy_to_borrow', 'fractionable'], inplace=True)
# active_assets_df.set_index(keys='id', drop=True, inplace=True)
display(active_assets_df.columns)

Index(['exchange', 'symbol'], dtype='object')

In [17]:
exchange_list = active_assets_df['exchange'].unique().tolist()
display(exchange_list)

['NYSE', 'NASDAQ', 'ARCA', 'AMEX', 'BATS', 'OTC']

In [18]:
today = dt.datetime.now()
offset = max(1, (today.weekday() + 6) % 7 -3)
timedelta = dt.timedelta(offset)
most_recent = today - timedelta
print(f"Most recent trading day: {most_recent.date()}")
numdays=365
oldest_date = most_recent - dt.timedelta(days=numdays)

def get_last_price(symbol):
    return alp.get_bars(
        symbol=symbol,
        timeframe='1D', 
        # limit=100, 
        end=str(most_recent.date()),
        start=str(oldest_date.date()),
    ).df

Most recent trading day: 2022-05-09


In [19]:
output = widgets.Output()
display(output)

Output()

In [20]:
stock_dict = {}
pop_set = {'OTC','BATS', 'ARCA'}
exchange_set = set(exchange_list)
exchange_set = exchange_set.difference(pop_set)
# display(list(exchange_set))
exchange_list = list(exchange_set)
periods_dict = {
    "pct_chg_24_hour": 1, 
    "pct_chg_1_week": 5,
    "pct_chg_1_month": 21,
    "pct_chg_3_months": 63,
    "pct_chg_6_months": 126,
    "pct_chg_1_year": 252,
    }
# display(periods_list)

for exchange in exchange_list:
    display(exchange)
    stock_list = list(active_assets_df.loc[lambda df: (df['exchange'] == exchange)]['symbol'])
    # display(stock_list)
    stock_dict[exchange] = pd.DataFrame()
    for i, symb in enumerate(stock_list[:]):
        sleep(.25)
        print(f"\r{i}/{len(stock_list)} {symb}", end='')
        # display(exchange, symb)
        bars_df = get_last_price(symb)
        drop_cols=['open', 'high', 'low', 'volume', 'trade_count', 'vwap']
        
        try:
            bars_df = pd.DataFrame(bars_df.loc[:, 'close'])
        except Exception as e:
            display(f"Exception {e} for {symb} bars_df")
        else:
            for k, v in periods_dict.items():
                temp_df = pd.DataFrame(bars_df.loc[:,'close'].pct_change(periods=v))
                temp_df.rename(columns={'close':k}, inplace=True)
                bars_df = bars_df.join(temp_df, how='outer')
            # stock_dict[exchange].iloc[symb] = bars_df.iloc[-1]
            price_hist_row = pd.DataFrame(bars_df.iloc[-1])
            price_hist_row.rename(columns={bars_df.index[-1]: symb}, inplace=True)
            # display(price_hist_row.T)
            # display(stock_dict[exchange])

        # display(bars_df.tail(1))
            try:
                stock_dict[exchange] = stock_dict[exchange].join(price_hist_row.T, how='outer')
            except ValueError:
                try:
                    stock_dict[exchange] = pd.concat([stock_dict[exchange], price_hist_row.T], join='outer')
                except Exception as e:
                    print(f"Exception: {e}")


        # stock_dict[exchange] = stock_dict[exchange].join(price_hist_row.T, how='outer')
        # display(stock_dict[exchange])
        # with output:
        #     # output.clear_output()
        #     try:
        #         display(f"{exchange}: {bars_df.index[0]} to {bars_df.index[-1]}")
        #     except Exception as e:
        #         pass
        #         display(exchange, e)
        #     display(bars_df.tail(1))
        # stock_dict[exchange] = 


'NASDAQ'

3578/4989 SOSHO

"Exception 'close' for SOSH bars_df"

3592/4989 WYTCU

"Exception 'close' for WYTC bars_df"

...

2338/2975 AXAC.RT

"Exception 'close' for AXAC.RT bars_df"

2530/2975 EVEXUU

"Exception 'close' for EVEX bars_df"

2974/2975 ASAB.U

In [23]:
display(exchange_list)

['NASDAQ', 'AMEX', 'NYSE']

In [26]:
display(stock_dict[exchange_list[0]])

,close,pct_chg_24_hour,pct_chg_1_week,pct_chg_1_month,pct_chg_3_months,pct_chg_6_months,pct_chg_1_year
BSJM,22.4700,-0.002663,-0.006192,-0.015769,-0.023468,-0.034379,-0.036449
BSJN,23.6400,-0.006305,-0.011706,-0.027961,-0.044462,-0.068558,-0.069291
BSJO,22.7200,-0.007427,-0.012603,-0.035244,-0.064636,-0.095181,-0.093738
BSJP,22.8200,-0.007395,-0.013403,-0.031820,-0.051143,-0.073111,-0.074615
BSJQ,23.2100,-0.011078,-0.016525,-0.040116,-0.067871,-0.098641,-0.096887
...,...,...,...,...,...,...,...
ACOR,0.8500,-0.122173,-0.351145,-0.484848,-0.554974,-0.780928,-0.796163
AEAEU,10.1200,0.000000,0.006965,0.005954,0.003968,0.001980,NaN
AEAE,9.9600,0.000000,-0.001003,0.002012,0.003021,NaN,NaN
AEACU,9.9000,-0.001009,-0.005025,-0.006523,-0.003021,-0.012961,-0.014925


In [27]:
display(stock_dict[exchange_list[1]])

,close,pct_chg_24_hour,pct_chg_1_week,pct_chg_1_month,pct_chg_3_months,pct_chg_6_months,pct_chg_1_year
BTG,4.03,-0.064965,-0.042755,-0.125813,0.089189,-0.077803,-0.201980
BTN,2.94,-0.020000,-0.016722,-0.039216,-0.006757,-0.045455,-0.379747
BTTR,1.91,-0.111628,-0.059113,-0.268199,-0.292593,-0.502604,NaN
CANF,0.84,-0.076923,-0.137134,-0.263158,-0.256637,-0.436242,-0.582090
CCF,84.90,0.009633,-0.001529,-0.004222,-0.123024,-0.202293,-0.263979
...,...,...,...,...,...,...,...
AGE,0.72,-0.162401,-0.168495,-0.202834,-0.121951,-0.121951,-0.446154
ASM,0.64,-0.080724,-0.085584,-0.296703,-0.137350,-0.335410,-0.518797
AMS,2.33,0.013043,0.044843,-0.004274,0.073733,-0.218121,-0.167857
AMPE,0.22,-0.048443,0.000000,-0.497028,-0.560878,-0.863354,-0.882979


In [28]:
display(stock_dict[exchange_list[2]])

,close,pct_chg_24_hour,pct_chg_1_week,pct_chg_1_month,pct_chg_3_months,pct_chg_6_months,pct_chg_1_year
BSIG,20.02,-0.011358,-0.010870,-0.099415,-0.155987,-0.339056,-0.120000
BSL,14.25,-0.022634,-0.013841,-0.072266,-0.124155,-0.151281,-0.100379
BSM,14.63,-0.049383,-0.077554,0.018093,0.224268,0.203125,0.359665
BSMX,5.40,-0.017002,0.001855,0.000000,-0.032258,-0.156250,-0.054291
BST,35.66,-0.052352,-0.078315,-0.154576,-0.208259,-0.358056,-0.357593
...,...,...,...,...,...,...,...
AIZ,180.97,-0.040354,0.000719,-0.025681,0.184824,0.113387,0.131982
BHP,62.14,-0.056340,-0.078042,-0.196639,-0.092184,0.163235,-0.231226
ASAI,14.28,-0.007644,-0.055556,-0.156028,0.169533,0.039301,-0.148987
BHLB,25.23,-0.006302,0.018982,-0.086862,-0.171156,-0.116597,0.024777


In [32]:
database_connection_string = 'sqlite:///test_db.db'
engine = sqlalchemy.create_engine(database_connection_string)
display(engine)


Engine(sqlite:///test_db.db)

In [37]:
display(stock_dict[exchange_list[0]].head())
for exchange in exchange_list:
    stock_dict[exchange].to_sql(exchange, engine)


,close,pct_chg_24_hour,pct_chg_1_week,pct_chg_1_month,pct_chg_3_months,pct_chg_6_months,pct_chg_1_year
BSJM,22.47,-0.002663,-0.006192,-0.015769,-0.023468,-0.034379,-0.036449
BSJN,23.64,-0.006305,-0.011706,-0.027961,-0.044462,-0.068558,-0.069291
BSJO,22.72,-0.007427,-0.012603,-0.035244,-0.064636,-0.095181,-0.093738
BSJP,22.82,-0.007395,-0.013403,-0.031820,-0.051143,-0.073111,-0.074615
BSJQ,23.21,-0.011078,-0.016525,-0.040116,-0.067871,-0.098641,-0.096887


In [40]:
display(sqlalchemy.inspect(engine).get_table_names())

['AMEX', 'NASDAQ', 'NYSE']